In [6]:
import pandas as pd
import numpy as np

from sklearn import preprocessing

# from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit
# from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.naive_bayes import BernoulliNB
from sklearn import svm

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [7]:
df = pd.read_csv('./data/processed_data/all/resample_to_50_per_act/time_features/time_w50.csv')
# df = pd.read_csv('./realtime_fixseg/feature_matrix_all.csv')
df.shape
df_data = np.array(df.drop('label',axis=1))
df_data.shape
df_labels = df['label']
df_labels.shape
train_data, test_data, train_labels, test_labels = train_test_split(df_data, df_labels, test_size=0.1)

(5026, 58)

(5026, 57)

(5026,)

### 1. kNN

In [8]:
#knn = KNeighborsClassifier(n_neighbors=5, weights='distance')
knn = KNeighborsClassifier()

In [9]:
# 对数据集进行指定次数的交叉验证并为每次验证效果评测
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
# cv = KFold(n_splits=20)
scores = cross_val_score(knn, df_data, df_labels, cv=cv)

In [10]:
scores

array([ 0.88071571,  0.91848907,  0.90854871,  0.88270378,  0.89264414,
        0.88667992,  0.89662028,  0.88071571,  0.89264414,  0.90457256])

In [11]:
scores.mean()

0.89443339960238577

In [12]:
knn.fit(train_data, train_labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [13]:
knn_labels = knn.predict(test_data)

In [14]:
knn_labels.size

503

In [15]:
test_labels.size

503

In [16]:
#knn_labels[0] == test_labels[0]

In [17]:
accuracy_score(test_labels, knn_labels)

0.90457256461232605

In [18]:
print(classification_report(test_labels, knn_labels,digits=5))

             precision    recall  f1-score   support

      bweep    0.86792   0.95172   0.90789       145
      clean    0.90323   0.83582   0.86822        67
      daily    0.93069   0.87037   0.89952       108
       dump    0.85366   0.87500   0.86420        40
        run    0.87500   0.63636   0.73684        11
      sweep    0.95495   0.94643   0.95067       112
       walk    0.90476   0.95000   0.92683        20

avg / total    0.90597   0.90457   0.90387       503



In [19]:
cm = confusion_matrix(test_labels, knn_labels)
cm_nor = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cm
cm_nor

array([[138,   2,   2,   1,   0,   1,   1],
       [  7,  56,   2,   1,   1,   0,   0],
       [ 10,   1,  94,   2,   0,   1,   0],
       [  0,   0,   1,  35,   0,   3,   1],
       [  2,   2,   0,   0,   7,   0,   0],
       [  2,   1,   1,   2,   0, 106,   0],
       [  0,   0,   1,   0,   0,   0,  19]], dtype=int64)

array([[ 0.95172414,  0.0137931 ,  0.0137931 ,  0.00689655,  0.        ,
         0.00689655,  0.00689655],
       [ 0.10447761,  0.8358209 ,  0.02985075,  0.01492537,  0.01492537,
         0.        ,  0.        ],
       [ 0.09259259,  0.00925926,  0.87037037,  0.01851852,  0.        ,
         0.00925926,  0.        ],
       [ 0.        ,  0.        ,  0.025     ,  0.875     ,  0.        ,
         0.075     ,  0.025     ],
       [ 0.18181818,  0.18181818,  0.        ,  0.        ,  0.63636364,
         0.        ,  0.        ],
       [ 0.01785714,  0.00892857,  0.00892857,  0.01785714,  0.        ,
         0.94642857,  0.        ],
       [ 0.        ,  0.        ,  0.05      ,  0.        ,  0.        ,
         0.        ,  0.95      ]])

### 2. DesicisionTree

In [20]:
dt = DecisionTreeClassifier(criterion='entropy')

In [21]:
# 对数据集进行指定次数的交叉验证并为每次验证效果评测
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
scores = cross_val_score(dt, df_data, df_labels, cv=cv)

In [22]:
scores

array([ 0.83697813,  0.8389662 ,  0.8389662 ,  0.84890656,  0.86282306,
        0.8389662 ,  0.82703777,  0.84493042,  0.83101392,  0.83101392])

In [23]:
scores.mean()

0.83996023856858848

In [24]:
dt.fit(train_data, train_labels)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [25]:
dt_labels = dt.predict(test_data)

In [26]:
accuracy_score(test_labels, dt_labels)

0.84294234592445327

In [27]:
print(classification_report(test_labels, dt_labels, digits=5))

             precision    recall  f1-score   support

      bweep    0.80519   0.85517   0.82943       145
      clean    0.88333   0.79104   0.83465        67
      daily    0.83654   0.80556   0.82075       108
       dump    0.80000   0.90000   0.84706        40
        run    0.80000   0.72727   0.76190        11
      sweep    0.90991   0.90179   0.90583       112
       walk    0.78947   0.75000   0.76923        20

avg / total    0.84450   0.84294   0.84281       503



### 3. Naive Bayes

In [28]:
gnb = GaussianNB()
#gnb = BernoulliNB()

In [29]:
# 对数据集进行指定次数的交叉验证并为每次验证效果评测
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
scores = cross_val_score(gnb, df_data, df_labels, cv=cv)

In [30]:
scores

array([ 0.58250497,  0.54671968,  0.59244533,  0.60437376,  0.5944334 ,
        0.55666004,  0.56461233,  0.57455268,  0.60039761,  0.60437376])

In [31]:
scores.mean()

0.5821073558648111

In [32]:
gnb.fit(train_data, train_labels)

GaussianNB(priors=None)

In [33]:
gnb_labels = gnb.predict(test_data)

In [34]:
accuracy_score(test_labels, gnb_labels)

0.53280318091451295

In [35]:
print(classification_report(test_labels, gnb_labels, digits=5))

             precision    recall  f1-score   support

      bweep    0.46535   0.32414   0.38211       145
      clean    0.48837   0.62687   0.54902        67
      daily    0.69231   0.25000   0.36735       108
       dump    0.59184   0.72500   0.65169        40
        run    0.50000   0.63636   0.56000        11
      sweep    0.59281   0.88393   0.70968       112
       walk    0.36170   0.85000   0.50746        20

avg / total    0.55222   0.53280   0.50442       503



### 4. SVM

In [36]:
clf = svm.SVC(C=5, cache_size=1000)

In [37]:
# 对数据集进行指定次数的交叉验证并为每次验证效果评测
# cv = ShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
# cv = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.1)
scores = cross_val_score(clf, df_data, df_labels, cv=cv)

In [38]:
scores

array([ 0.91252485,  0.92644135,  0.92445328,  0.93240557,  0.92644135,
        0.91848907,  0.94234592,  0.91848907,  0.90258449,  0.91848907])

In [39]:
scores.mean()

0.92226640159045714

In [40]:
# T = 10
# ave_scores =[]
# for t in range(T):
#     ave_scores.append( cross_val_score(clf, df_data, df_labels, cv=cv).mean() )
# ave_scores = np.array(ave_scores)
# ave_scores
# ave_scores.mean()

In [41]:
clf.fit(train_data, train_labels)

SVC(C=5, cache_size=1000, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [42]:
#这个计算出的实际上是recall的加权平均值
clf.score(test_data, test_labels)

0.91451292246520877

In [43]:
clf_labels =clf.predict(test_data)

In [44]:
accuracy_score(test_labels, clf_labels)  

0.91451292246520877

In [45]:
#这个计算的是f1-score的加权平均值
f1_score(test_labels, clf_labels, average='weighted')  

0.91396843069919087

In [46]:
print(classification_report(test_labels, clf_labels,digits=5))

             precision    recall  f1-score   support

      bweep    0.91216   0.93103   0.92150       145
      clean    0.91045   0.91045   0.91045        67
      daily    0.89091   0.90741   0.89908       108
       dump    0.87179   0.85000   0.86076        40
        run    0.87500   0.63636   0.73684        11
      sweep    0.94690   0.95536   0.95111       112
       walk    1.00000   0.90000   0.94737        20

avg / total    0.91458   0.91451   0.91397       503

